## Agents on Kubeflow 🤓

In this tutorial we will be training a reinforcement learning agent from the [tensorflow/agents](https://github.com/tensorflow/agents) project on Kubernetes using [Kubeflow](https://github.com/google/kubeflow).

The task the agent will be learning to perform is to operate a Kuka Robotics arm simulated in the OpenAI Gym Bullet Physics 'KukaBulletEnv-v0' environment. Feel free to [skip to the end](http://localhost:8888/notebooks/kubeflow-rl/apps/agents_ppo/demo.ipynb#Rendering-the-model) to see what this will look like!

### Setup

This demo assumes you've followed the setup steps provided in the readme. Beyond that the only setup needed here is to set the "BUCKET" variable to the same name that was used in those setup steps (or any other bucket accessible to the currently set and authorized GCloud project):

In [117]:
# Bucket to use
BUCKET="kubeflow-rl"

### Training

The objective of the training phase is to learn the parameterization of our model that confers a high level of performance on the provided task. Here we'll launch and monitor a job.

#### Launching the TFJob

We'll use [ksonnet](https://ksonnet.io/) to parameterize and apply a TFJob configuration (i.e. run a job). Here you can change the image to be a custom job image, such as one built and deployed with build.sh, or use the one provided here if you only want to change parameters. Below we'll display the templated job YAML for reference.

In [ ]:
# Check your cluster and see if that matches one of the existing ksonnet environments
# You want the kubernetes master server to be the same as the server listed for the ks environment
!kubectl cluster-info
!ks env list

In [108]:
image = "gcr.io/kubeflow-rl/agents:tf1.4.1-0208-0959-a276"

study = {"name": "replicated-env-comparison",
         "experiments": [{"name": "kuka",
                          "image": image,
                          "env": "KukaBulletEnv-v0",
                          "num_replicas": 4},
                         {"name": "pendulum",
                          "image": image,
                          "env": "InvertedPendulumBulletEnv-v0",
                          "num_replicas": 4}
                        ]
        }


STUDY_LOGS_ROOT = "gs://{0}/studies/{1}".format(BUCKET, study["name"])

print(STUDY_LOGS_ROOT)

gs://kubeflow-rl-kf/studies/replicated-env-comparison


In [ ]:
import datetime
import uuid
import os
import pprint

os.chdir(APP_ROOT)

print("Preparing study: %s..." % study["name"])

for experiment in study["experiments"]:
    
    print("Preparing experiment: %s" % experiment["name"])
    
    # Get and set the job container image for this experiment
    IMAGE = experiment["image"]
    !ks param set agents image {IMAGE}

    # Set the gym learning environment on which to train
    ENVIRONMENT = experiment["env"]
    !ks param set agents env {ENVIRONMENT}

    # Set the algorithm and network part to use for policy and value networks
    !ks param set agents algorithm "agents.ppo.PPOAlgorithm"
    !ks param set agents network "agents.scripts.networks.feed_forward_gaussian"

    # Run in training mode with 30 CPU and 30 agents for 20M steps
    !ks param set agents run_mode train
    !ks param set agents num_cpu 30
    !ks param set agents num_agents 30
    !ks param set agents steps 15e6

    !ks param set agents update_every 60
    !ks param set agents eval_episodes 25

    for replica_id in range(experiment["num_replicas"]):

        # Construct a unique name for the training job based on experiment["name"]
        now=datetime.datetime.now()
        JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
        BASE_NAME = experiment["name"]
        TRAIN_JOB_NAME=BASE_NAME + "-" + JOB_SALT
        !ks param set agents name {TRAIN_JOB_NAME}

        # Construct a log dir path for this experiment
        LOG_DIR="{0}/{1}".format(STUDY_LOGS_ROOT, TRAIN_JOB_NAME)
        !ks param set agents log_dir {LOG_DIR}

        if "replicas" not in experiment:
            experiment["replicas"] = []
        experiment["replicas"].append({"log_dir": LOG_DIR})
        
        print("Preparing replica %s of %s for experiment %s" % (replica_id + 1, experiment["replicas"], experiment["name"]))
        !ks apply gke -c agents


In [110]:
study

{'experiments': [{'env': 'KukaBulletEnv-v0',
   'image': 'gcr.io/kubeflow-rl/agents:tf1.4.1-0208-0959-a276',
   'name': 'kuka',
   'num_replicas': 4,
   'replicas': [{'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-10df'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-7964'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-922d'}]},
  {'env': 'InvertedPendulumBulletEnv-v0',
   'image': 'gcr.io/kubeflow-rl/agents:tf1.4.1-0208-0959-a276',
   'name': 'pendulum',
   'num_replicas': 4,
   'replicas': [{'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/pendulum-0208-1010-5e21'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/pendulum-0208-1010-9de4'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/pendulum-0208-1010-7b7f'},
   

Now we can list tfjobs and see that a job has been created.

In [29]:
!kubectl get tfjobs -n {NAMESPACE}

No resources found.


#### Monitoring training

The IDs, status, and other metadata of pods involved in the training job can be displayed using the following:

In [ ]:
!kubectl get pods -n rl --show-all

In [46]:
TRAIN_JOB_NAME

'kuka-mts-tiny-0206-1404-ae16'

Obtain the ID of the master pod and print logs

In [ ]:
import subprocess
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + TRAIN_JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl logs -n {NAMESPACE} {master_pod}

In [ ]:
## Tensorboard

#### Launching tensorboard

The following will create a tensorboard deployment using the STUDY_LOGS_ROOT as the source of tensorboard logs.

In [ ]:
HPARAM_SET="tboard"
now=datetime.datetime.now()
JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
TBOARD_JOB_NAME=HPARAM_SET + "-" + JOB_SALT
SECRET_NAME="gcp-credentials"
SECRET_FILE_NAME="secret.json"
NAMESPACE="rl"

!ks param set tensorboard name {TBOARD_JOB_NAME}
!ks param set tensorboard namespace {NAMESPACE}
!ks param set tensorboard log_dir {STUDY_LOGS_ROOT}
!ks param set tensorboard secret {SECRET_NAME}
!ks param set tensorboard secret_file_name {SECRET_FILE_NAME}
!ks show default -c tensorboard

!ks apply gke -c tensorboard

### Connecting to Tensorboard

To connect to tensorboard start the kubernetes proxy with `kubectl proxy` then access Tensorboard via the url given by evaluating the next cell

In [99]:
PROXY_PORT=8001
# url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
#     proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=TRAIN_JOB_NAME + "-tb"))
url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
    proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=TBOARD_JOB_NAME + "-tb"))
print(url)

http://127.0.0.1:8001/api/v1/proxy/namespaces/rl/services/tboard-0208-1533-c315-tb:80/


Below is a screen grab from Tensorboard of the mean_score for four identically parameterized workers training on the Kuka environment:

![](tboard_mean_score.png)

As you can see there is variability both between and within workers but some still seem to reach stably high performance.

## Rendering!

Let's take a look at the trained model performing the task!!

#### Initiating render jobs

Here we'll create a render job for each of the experiments in our study. This will generate multiple MP4 videos of the agent performing the task for each replica run with renders landing in ${LOG_DIR}/render/[some_unique_subdir_name].

In [ ]:
import datetime
import uuid
import os

os.chdir(APP_ROOT)

for experiment in study["experiments"]:
    for replica in experiment["replicas"]:
        
        LOG_DIR = replica["log_dir"]
        IMAGE = experiment["image"]
        
        now=datetime.datetime.now()

        JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
        RENDER_JOB_NAME="render-" + JOB_SALT
        !ks param set agents_render name {RENDER_JOB_NAME}

        !ks param set agents_render log_dir {LOG_DIR}
        !ks param set agents_render image {IMAGE}
        
        !ks apply gke -c agents_render

It will take about 15min for each render to run. We can see whether the renders are complete by checking the third column in the following:

In [ ]:
!kubectl get jobs -n rl --show-all

First let's get the GCS path of the directory containing renders for an experiment of interest. For a reminder, here's the structure of our study:

In [111]:
study

{'experiments': [{'env': 'KukaBulletEnv-v0',
   'image': 'gcr.io/kubeflow-rl/agents:tf1.4.1-0208-0959-a276',
   'name': 'kuka',
   'num_replicas': 4,
   'replicas': [{'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-10df'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-7964'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-922d'}]},
  {'env': 'InvertedPendulumBulletEnv-v0',
   'image': 'gcr.io/kubeflow-rl/agents:tf1.4.1-0208-0959-a276',
   'name': 'pendulum',
   'num_replicas': 4,
   'replicas': [{'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/pendulum-0208-1010-5e21'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/pendulum-0208-1010-9de4'},
    {'log_dir': 'gs://kubeflow-rl-kf/studies/replicated-env-comparison/pendulum-0208-1010-7b7f'},
   

Let's take a look at the renders for the first replica of the first experiment. First let's take a look at the log dir and render/ subdir of that experiment:

In [112]:
kuka_experiment_logdir = study["experiments"][0]["replicas"][0]["log_dir"]
!gsutil ls {kuka_experiment_logdir}
!gsutil ls {kuka_experiment_logdir}/render

gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/checkpoint
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/config.yaml
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/model.ckpt-17850000.data-00000-of-00001
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/model.ckpt-17850000.index
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/model.ckpt-17850000.meta
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/model.ckpt-18700020.data-00000-of-00001
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/model.ckpt-18700020.index
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/model.ckpt-18700020.meta
gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/model.ckpt-19550010.data-00000-of-00001
gs://kubeflow-

Now let's specify exactly which render directory from which to pull a render for inspection (in this case we only have one available):

In [113]:
RENDER_DIR="gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/render/0208-2228-7d1d/"

Now let's take a look at one of those renders:

In [114]:
!mkdir -p /tmp/agents-render
!gsutil cp `gsutil ls {RENDER_DIR} | grep mp4 | head -n1` /tmp/agents-render/render.mp4

Copying gs://kubeflow-rl-kf/studies/replicated-env-comparison/kuka-0208-1010-4f3a/render/0208-2228-7d1d/openaigym.video.0.45.video000000.mp4...
- [1 files][210.8 KiB/210.8 KiB]                                                
Operation completed over 1 objects/210.8 KiB.                                    


#### Inspecting the result

When the job is complete there will be a subdirectory of the log dir named "render" with a number of short videos of episodes of the agent performing the grasping task. Here's an example of what one of those looks like in a well-trained model.

In [116]:
import io
import base64
from IPython.display import HTML

mp4_path = '/tmp/agents-render/render.mp4'

video = io.open(mp4_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

### Great job! 🎉🎉🎉

If this is your first time working with these technologies you might be interested in some suggestions of good next steps. Here are some ideas:
- Try training with some other learning environments (from the ID fields [here](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/__init__.py)) and tweet your results! E.g.
    - RacecarBulletEnv-v0
    - MinitaurBulletDuckEnv-v0
    - HalfCheetahBulletEnv-v0
- Take a shot at implementing your own gym learning environment and repeat the above.